In [1]:
import tensorflow as tf
import datetime
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

with tf.device("/device:GPU:0"):
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Creates a session with log_device_placement set to True.
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20001):
    batch = mnist.train.next_batch(50)
    if i % 1000 == 0:
        print(datetime.datetime.now())
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


2017-12-23 04:56:26.310446
step 0, training accuracy 0.12
2017-12-23 04:56:32.224935
step 1000, training accuracy 0.94
2017-12-23 04:56:38.003365
step 2000, training accuracy 0.96
2017-12-23 04:56:43.845726
step 3000, training accuracy 0.94
2017-12-23 04:56:49.693206
step 4000, training accuracy 0.98
2017-12-23 04:56:55.607407
step 5000, training accuracy 0.98
2017-12-23 04:57:01.512869
step 6000, training accuracy 1
2017-12-23 04:57:07.479537
step 7000, training accuracy 0.98
2017-12-23 04:57:13.427878
step 8000, training accuracy 0.98
2017-12-23 04:57:19.221202
step 9000, training accuracy 1
2017-12-23 04:57:25.217994
step 10000, training accuracy 0.96
2017-12-23 04:57:31.123012
step 11000, training accuracy 0.98
2017-12-23 04:57:37.020662
step 12000, training accuracy 1
2017-12-23 04:57:42.833898
step 13000, training accuracy 1
2017-12-23 04:57:48.804130
step 14000, training accuracy 1
2017-12-23 04:57:54.648924
step 15000, training accuracy 0.98
2017-12-23 04:58:00.446129
step 1600

9.898305084745763
